## Obtendo Eto para grama utilizando Google Earth Engine.

### Código preparado para funcionar com dados de reanálises do era5 e gldas.
#### Dados de entrada: 
##### - min_data, max_data, tabela_estacoes;
#### Dados de saída: 
##### - tabela (exportada para o Drive).

In [1]:
from meteorology import meteorology, eto_grass
from datetime import datetime, date, timedelta
import pandas as pd

In [2]:
# Importando a biblioteca do Google Earth Engine
import ee

In [3]:
try:
    ee.Initialize()
except ee.EEException:
    ee.Authenticate()
    ee.Initialize()

In [4]:
'''
Dados de entrada
    min_data: 
        Data inicial.
    max_data:
        Data final.
    table_towers:
        Dados de longitude, latitude e nome da estação automática.
'''
# Selecionar o intervalo das datas FMT: YYYY-MM-DD
min_data = ee.Date('2004-01-01') 
max_data = ee.Date('2019-12-31')

# tabela com informações das torres automaticas
table_towers = pd.read_csv(r'D:\mestrado\eto_ana\estacoes_automaticas_eto.csv', delimiter=';',
                           sep='delimiter', engine='python')

# Renomeando as colunas da tabela
table_towers['latitude'] = table_towers['ein_latitude']
table_towers['longitude'] = table_towers['ein_longitude']
table_towers['station']=table_towers['ï»¿ein_estacao_cd']

In [5]:
# Modelo Digital de Elevacao
elev_source = 'USGS/SRTMGL1_003'

# Criando lista de datas
n_months = max_data.difference(min_data,'day').round();
dates = ee.List.sequence(0,n_months,1);
def make_datelist(n):
  return min_data.advance(n,'day');

dates = dates.map(make_datelist)

# Funcao para calcular a ETo
def calc_eto(time_start):
    time_start = ee.Date(time_start)
    tmin, tmax, tair, rso24h, ws = meteorology(time_start, "NASA/GLDAS/V021/NOAH/G025/T3H")
    etodata = eto_grass(time_start, tmin, tmax, tair, ws, rso24h, elev_source)
    
    tmin, tmax, tair, rso24h, ws = meteorology(time_start, "ECMWF/ERA5_LAND/HOURLY")
    eto_data_era5 = eto_grass(time_start, tmin, tmax, tair, ws, rso24h, elev_source)
    eto_data_era5 = eto_data_era5.select('eto24h').rename('eto24h_era5')
    
    etodata = etodata.addBands(eto_data_era5).set('system:time_start',time_start.millis())
    return etodata

eto_col = ee.ImageCollection.fromImages(
    dates.map(calc_eto)
)

In [6]:
# Funcao que exporta uma tabela com os resultados de ETO para cada estacao
#da tabela inseridada
for coords in zip(table_towers['longitude'],
                            table_towers['latitude'],
                            table_towers['station']):
    
    point = ee.Geometry.Point([coords[0],coords[1]])
    
    # apresenta no console longitude,latitude e estacao rodada.
    print('longitude',coords[0])
    print('latitude',coords[1])
    print(coords[2])
    def extract_ts(image):
        
        red = image.reduceRegion(
        reducer=ee.Reducer.mean(),
        scale = 25000,
        maxPixels = 10e12,
        geometry = point
        )
    
        date= ee.Date(image.get('system:time_start')).format('YYYY-MM-dd')
        return ee.Feature(None, {'eto_gldas': ee.List([red.get('eto24h'),-9999])\
                                         .reduce(ee.Reducer.firstNonNull()),
                                 'eto_era5': ee.List([red.get('eto24h_era5'),-9999])\
                                         .reduce(ee.Reducer.firstNonNull()),
                                           'date': date})
    
    table_eto = eto_col.map(extract_ts)
    
    task = ee.batch.Export.table.toDrive(**{
      'collection': table_eto,
      'folder': 'eto_gldas',
      'description':coords[2].lower()+'_eto_gldas_era5',
      'fileFormat': 'CSV',
      'selectors': ['date','eto_gldas','eto_era5']
    })
    task.start()
    

longitude -47.925756
latitude -15.789342999999999
A001
longitude -49.220222
latitude -16.642841
A002
